In [1]:
import os

In [2]:
%pwd

'd:\\Nikku\\End to End Project\\Healthy.ai\\research\\Diet_Recommendation'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\Nikku\\End to End Project\\Healthy.ai'

## Entity

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DietRecomdModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_name: str
    n_clusters: int
    target_column: float

## Configuration Manager

In [7]:
from Healthyai.constants import *
from Healthyai.utils.common import read_yaml, create_directories


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts])
        
    def diet_recommend_model_trainer_config(self) -> DietRecomdModelTrainerConfig:
        config = self.config.diet_recommendation_model_trainer
        params = self.params.KMeans
        schema = self.schema.COLUMNS
        
        
        create_directories([config.root_dir])
        
        
        model_trainer_config = DietRecomdModelTrainerConfig(
            root_dir=config.root_dir,
            data_path = config.data_path,
            model_name= config.model_name,
            n_clusters=params.n_clusters,
            target_column=schema.Calories
        )
        return model_trainer_config

## Components

In [9]:
import pandas as pd
import numpy as np
import os
from Healthyai import logger
from sklearn.cluster import KMeans
import joblib


In [10]:
class DietModelTrainer:
    def __init__(self, config: DietRecomdModelTrainerConfig):
        self.config = config
        
    def train(self):
        data = pd.read_csv(self.config.data_path, index_col=0)
        # print(data.head())
        # print(data['Calories'])
        train_data = data["Calories"]
        
        kmeans = KMeans(n_clusters=self.config.n_clusters, random_state= 42)
        kmeans.fit(np.array(train_data).reshape(-1,1))
        
        joblib.dump(kmeans,os.path.join(self.config.root_dir, self.config.model_name))
        
        

## pipeline

In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.diet_recommend_model_trainer_config()
    model_trainer_config = DietModelTrainer(config = model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-02-03 12:18:05,367: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-03 12:18:05,369: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-03 12:18:05,375: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-03 12:18:05,376: INFO: common: Created directory at : artifacts]
[2024-02-03 12:18:05,376: INFO: common: Created directory at : artifacts/diet_recomendation/model_trainer]


c:\Users\akash\anaconda3\envs\healthyai\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\akash\anaconda3\envs\healthyai\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\akash\anaconda3\envs\healthyai\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


## Testing

In [12]:
model = joblib.load('artifacts/diet_recomendation/model_trainer/model.joblib')

In [22]:
df = pd.read_csv("artifacts/diet_recomendation/data_ingestion/diet_recommendation.csv")

In [23]:
df.head()

,Breakfast,Lunch,Snacks,Dinner,Calories,Carbs,Total Fat,Saturated Fat,Protein,Fiber,...,Magnesium,Phosphorus,Vitamin C,Vitamin A,Calcium,Iron,Zinc,Vitamin E,Vitamin K,food_items
0,1,0,1,0,94.64,25.48,0.364,0.182,0.546,4.368,...,9.10,20.02,8.372,10.92,0.3276,4.004,10.92,0.1820,0.0910,Apple
1,1,0,1,0,26.40,6.05,0.220,0.000,0.770,1.100,...,5.50,12.65,5.500,1059.30,0.4895,1.705,7.15,0.2145,0.1100,Apricot
2,1,0,1,0,48.00,2.70,0.600,0.090,0.600,0.600,...,8.70,15.60,3.000,2.10,0.6210,6.300,3.60,0.1800,0.1800,Avocado
3,1,0,1,0,105.02,27.14,0.354,0.118,1.298,3.068,...,31.86,25.96,10.266,1.18,0.1180,0.590,5.90,0.3540,0.1770,Banana
4,1,0,1,0,71.50,17.16,0.715,0.000,1.430,4.290,...,41.47,42.90,41.899,1258.40,2.4310,1.287,77.22,2.2880,1.0868,Black Chokeberry


In [24]:
cluster = model.labels_

In [25]:
df['cluster'] = cluster

In [26]:
df.head()

,Breakfast,Lunch,Snacks,Dinner,Calories,Carbs,Total Fat,Saturated Fat,Protein,Fiber,...,Phosphorus,Vitamin C,Vitamin A,Calcium,Iron,Zinc,Vitamin E,Vitamin K,food_items,cluster
0,1,0,1,0,94.64,25.48,0.364,0.182,0.546,4.368,...,20.02,8.372,10.92,0.3276,4.004,10.92,0.1820,0.0910,Apple,0
1,1,0,1,0,26.40,6.05,0.220,0.000,0.770,1.100,...,12.65,5.500,1059.30,0.4895,1.705,7.15,0.2145,0.1100,Apricot,4
2,1,0,1,0,48.00,2.70,0.600,0.090,0.600,0.600,...,15.60,3.000,2.10,0.6210,6.300,3.60,0.1800,0.1800,Avocado,4
3,1,0,1,0,105.02,27.14,0.354,0.118,1.298,3.068,...,25.96,10.266,1.18,0.1180,0.590,5.90,0.3540,0.1770,Banana,0
4,1,0,1,0,71.50,17.16,0.715,0.000,1.430,4.290,...,42.90,41.899,1258.40,2.4310,1.287,77.22,2.2880,1.0868,Black Chokeberry,0


In [31]:
breakfast_df = df.loc[df['Breakfast']==1,['food_items','Calories','cluster']]

In [32]:
def divide_calories(calories):
  breakfast = int(0.30 * calories)
  lunch = int(0.25 * calories)
  snacks = int(0.15 * calories)
  dinner = int(0.27 * calories)
  return breakfast,lunch,snacks,dinner

def cluster(new_calories):
  new = np.array(new_calories).reshape(-1,1)
  predict = model.predict(new)
  return predict[0]

def breakfastdf_extaction(df):
  breafastdf = df.loc[df['Breakfast']==1,['food_items','Calories','cluster']]
  return breafastdf
def lunchdf_extaction(df):
  lunchdf = df.loc[df['Lunch']==1,['food_items','Calories','cluster']]
  return lunchdf
def sanckdf_extaction(df):
  sanckdf = df.loc[df['Snacks']==1,['food_items','Calories','cluster']]
  return sanckdf
def dinnerdf_extaction(df):
  dinnrtdf = df.loc[df['Dinner']==1,['food_items','Calories','cluster']]
  return dinnrtdf




def fetch_recommendation(df,cluster):
  # df_lunch[(df_lunch['Cluster']==1) & (df_lunch['Calories']>=405) & (df_lunch['Calories']<=500)]
  final = df[df['cluster']==cluster].sort_values(by='Calories').head(4)
  return final['food_items'].to_list()



# weight loss

def weight_loss(calories):

  weight_loss = calories - 500

  # print('Weight calories',weight_loss)

  breakfastcal = divide_calories(weight_loss)[0]
  # print(breakfastcal)
  lunchcal = divide_calories(weight_loss)[1]
  # print(lunchcal)
  sancktcal = divide_calories(weight_loss)[2]
  # print(sancktcal)
  dinnercal = divide_calories(weight_loss)[3]
  # print(dinnercal)

  breakfastdf = breakfastdf_extaction(df)
  lunchdf = lunchdf_extaction(df)
  snackdf = sanckdf_extaction(df)
  dinnerdf = dinnerdf_extaction(df)


  beakclusters = cluster(breakfastcal)
  # print(beakclusters)
  lunchclusters = cluster(lunchcal)
  # print(lunchclusters)
  snackclusters = cluster(sancktcal)
  # print(snackclusters)
  dinnerclusters = cluster(dinnercal)
  # print(dinnerclusters)
  meals = []
  breakfastfood = fetch_recommendation(breakfastdf,beakclusters)
  meals.append(breakfastfood)
  lunchfood = fetch_recommendation(lunchdf,lunchclusters)
  meals.append(lunchfood)
  snackfood = fetch_recommendation(snackdf,snackclusters)
  meals.append(snackfood)
  dinnerfood = fetch_recommendation(dinnerdf,dinnerclusters)
  meals.append(dinnerfood)

  meal_categories = ['breakfast', 'lunch', 'snacks', 'dinner']

  meal_dict = {category: meal_list for category, meal_list in zip(meal_categories, meals)}

  #breakfastfood ,lunchfood,snackfood,dinnerfood
  return meal_dict


In [33]:
weight_loss(1700)

{'breakfast': ['Pomegranate Juice',
  'Chai',
  'Whole grain Sorghum',
  'Whole Grain Wheat'],
 'lunch': ['Pesto pasta salad', 'Pumpernickel', 'Mutton Meat', 'Hilsa Fish'],
 'snacks': ['Cashews', 'Jackfruit', 'Rose Water Lemonade', 'Cedar Nuts'],
 'dinner': ['Pesto pasta salad', 'Pumpernickel', 'Mutton Meat', 'Hilsa Fish']}

In [ ]:
d = weight_loss(1700)


In [ ]:
from box import ConfigBox

In [ ]:
data = ConfigBox(d)

In [ ]:
print(data.breakfast)
print(data.lunch)
print(data.snacks)
print(data.dinner)

['Pesto pasta salad', 'Pumpernickel', 'Mutton Meat', 'Hilsa Fish']
['Pomegranate Juice', 'Chai', 'Whole grain Sorghum', 'Whole Grain Wheat']
['Cashews', 'Jackfruit', 'Rose Water Lemonade', 'Cedar Nuts']
['Pesto pasta salad', 'Pumpernickel', 'Mutton Meat', 'Hilsa Fish']
